In [1]:
import findspark
findspark.init('/usr/local/spark/spark-2.4.0-bin-hadoop2.7')
import pyspark

import random

sc = pyspark.SparkContext(appName = 'test')

3.14147936


In [ ]:
# calculate pi value
num_samples = 100000000
def inside(p):
    x,y = random.random(), random.random()
    return x*x + y*y < 1
count = sc.parallelize(range(0,num_samples)).filter(inside).count()
pi = 4 * count/num_samples
print(pi)

In [2]:
from pyspark import SQLContext
from pyspark.sql import Row
import os

sqlContext = SQLContext(sc)
l = [('Ankit',25),('Jalfaizy',22),('saurabh',20),('Bala',26)]
rdd = sc.parallelize(l)
people = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))
People_df = sqlContext.createDataFrame(people)
People_df.show()
if os.path.exists('people.csv'):
    print('CSV File Exists Already!')
else:
    People_df.coalesce(1).write.csv('people.csv', header=True)
    print('CSV File Saving Success!')

+---+--------+
|age|    name|
+---+--------+
| 25|   Ankit|
| 22|Jalfaizy|
| 20| saurabh|
| 26|    Bala|
+---+--------+

CSV File Saving Success!


In [ ]:
from pyspark import SQLContext
#from pyspark.sql import Row
import os

sqlContext = SQLContext(sc)

if os.path.exists('people.csv'):
    sample_df = sqlContext.read.csv('people.csv', inferSchema=True, header=True)
    sample_df.show()
    print('CSV File Loading Success!')
else:
    print('CSV File Doesnot Exist!')

In [ ]:
sample_df.show(2)
#sample_df.head(2)
sample_df.printSchema() #show schema
sample_df.count()       #row count
sample_df.columns       #show colum names
sample_df.describe('age').show() #column profiling
sample_df.select('age').show() # select statement
sample_df.select('name').distinct().show() #select distinct statement
print(sample_df.select('name').distinct().count()) #count distinct statement
sample_df.select('age','name').dropDuplicates().show() #remove duplicates
print(sample_df.dropna().count()) #drop blank rows
sample_df.fillna(-1).show()       #fill blank row with default value
sample_df.filter(sample_df.age > 25).show() #filter/where statement
sample_df.groupby('name')\
.sum()\
.show()    #group by sum statement
sample_df.groupby('name')\
.count()\
.show()  #group by count statement
t1 = sample_df.sample(False, 0.2, 1)      #make another sample dataframe
t1.show()
rdd = sample_df.select('age')\
.rdd\
.map(lambda x:(x,1)) #transfer dataframe to rdd
for x in rdd.collect():
    print(x)
    
sample_df.orderBy(sample_df.age.desc()).show() #sort the rows
sample_df.withColumn('new column', sample_df.age/2).show() #add new column
sample_df.drop('age').show() #drop column

In [ ]:
#use SQL script to do the jobs
sample_df.registerTempTable('sample_table')
sqlContext.sql('select age, name, age/2 as new_column from sample_table order by age desc').show()

In [ ]:
sample_df.select('name','age').orderBy('age').withColumn('new column', sample_df.age/2).show()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import os

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

sc = spark.sparkContext    
    
#sqlContext = SQLContext(sc)

l = [('Ankit',25),('Jalfaizy',22),('saurabh',20),('Bala',26)]
rdd = sc.parallelize(l)
people = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))
People_df = spark.createDataFrame(people)
People_df.show()
if os.path.exists('people.csv'):
    print('CSV File Exists Already!')
else:
    People_df.coalesce(1).write.csv('people.csv', header=True)
    print('CSV File Saving Success!')
    
if os.path.exists('people.csv'):
    sample_df = spark.read.load('people.csv', format="csv", sep=",", inferSchema=True, header=True)
    sample_df.show()
    print('CSV File Loading Success!')
else:
    print('CSV File Does not Exist!')
    


In [ ]:
sample_df.write.option("path", "test").saveAsTable("t")

In [ ]:
#sc.stop()